In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pylab as plt
import warnings
import scipy
from scipy import stats
from sympy import symbols, diff
import math
%matplotlib inline
from scipy.optimize import fsolve

###  1. Провести дисперсионный анализ для определения того, есть ли различия среднего роста среди взрослых футболистов, хоккеистов и штангистов. Даны значения роста в трех группах случайно выбранных спортсменов:

##### Футболисты: 173, 175, 180, 178, 177, 185, 183, 182.
##### Хоккеисты: 177, 179, 180, 188, 177, 172, 171, 184, 180.
##### Штангисты: 172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170.

Сформулируем гипотезу: вид спорта не оказывает статистически значимого эффекта, т.е. не влияет на рост спортсменов

In [14]:
foot = np.array([173, 175, 180, 178, 177, 185, 183, 182], dtype=np.float64)
hock = np.array([177, 179, 180, 188, 177, 172, 171, 184, 180], dtype=np.float64)
athl = np.array([172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170], dtype=np.float64)

In [15]:
n1 = len(foot)
n2 = len(hock)
n3 = len(athl)
n = n1 + n2 + n3
print(f'Общее количество спортсменов: {n}')

Общее количество спортсменов: 28


In [16]:
# Количество групп
k = 3

Проведем однофакторный дисперсионный анализ. Найдем средний рост для каждой группы спортсменов.

In [17]:
# футболисты
foot_mean = np.mean(foot)
foot_mean

179.125

In [18]:
# хоккеисты
hock_mean = np.mean(hock)
hock_mean

178.66666666666666

In [19]:
# тяжелоатлеты
athl_mean = np.mean(athl)
athl_mean

172.72727272727272

Заметно, что средний рост для каждой группы спортсменов разнятся. Установим, что это отличие статистически значимо. Для этого сначала соберем все значения роста в один массив:

In [20]:
height_all = np.concatenate([foot, hock, athl])
height_all

array([173., 175., 180., 178., 177., 185., 183., 182., 177., 179., 180.,
       188., 177., 172., 171., 184., 180., 172., 173., 169., 177., 166.,
       180., 178., 177., 172., 166., 170.])

Найдем среднее значение роста спортсменов по всем значениям:

In [21]:
height_mean = np.mean(height_all)
height_mean

176.46428571428572


Найдем $S^2$ — сумму квадратов отклонений наблюдений от общего среднего:

In [22]:
s2 = np.sum((height_all - height_mean)**2)
s2

830.9642857142854

Найдем $S^2_F$ - сумму квадратов отклонений средних групповых значений от общего среднего:

In [23]:
s2_f = ((foot_mean - height_mean)**2) * n1 + ((hock_mean - height_mean)**2) * n2 + ((athl_mean - height_mean)**2) * n3
s2_f

253.9074675324678

Найдем $S^2_{ост}$ — остаточную сумму квадратов отклонений:

In [25]:
s2_residual = ((foot - foot_mean)**2).sum() + ((hock - hock_mean)**2).sum() + ((athl - athl_mean)**2).sum()
s2_residual

577.0568181818182

Удостоверимся, что соблюдается равенство $S^2 = S_F^2 + S_{ост}^2$:

In [26]:
print(s2)
print(s2_f + s2_residual)

830.9642857142854
830.964285714286


Найдем общую дисперсию:

In [27]:
sigma2_general = s2 / (n - 1)
sigma2_general

30.776455026455015

Найдем факторную дисперсию:

In [29]:
sigma2_f = s2_f / (k - 1)
sigma2_f

126.9537337662339

Найдем остаточную дисперсию:

In [30]:
sigma2_residual = s2_residual / (n - k)
sigma2_residual

23.08227272727273

Вычислим $F_H$:

In [31]:
F_h = sigma2_f / sigma2_residual
F_h

5.500053450812598

Найдем значение $F_{крит}$ в таблице критических точек распределения Фишера-Снедекора для заданного уровня значимости $\alpha = 0.05$ и двух степеней свободы:

$df_{межд} = k - 1 = 3 - 1 = 2$ и $df_{внутр} = n - k = 28 - 3 = 25$.

Для данных значений $F_{крит} = 3.38$. Так как $F_H > F_{крит}$, различие среднего роста среди взрослых футболистов, хоккеистов и штангистов статистически значимо.

Также вычислим эмпирическое корреляционное отношение  $η^2$ :

In [33]:
eta2 = np.sqrt(s2_f / s2)
eta2

0.5527726636647112

Для качественной оценки тесноты связи на основе показателя эмпирического корреляционного отношения можно воспользоваться соотношениями Чэддока: 0,1-0,3 (слабая связь), 0,3-0,5 (умеренная связь), 0,5-0,7 (заметная связь), 0,7-0,9 (тесная связь), 0,9-0,99 (весьма тесная связь).

В нашем случае значение  η=0.55, что означает, что связь роста между группами спортсменов заметная.

Проверим данные расчеты с помощью библотеки питон:

In [34]:
stats.f_oneway(foot, hock, athl)

F_onewayResult(statistic=5.500053450812596, pvalue=0.010482206918698694)

Мы видим, что статистика рассчитана правильно. Кроме того, также видно, что $alpha$ > $p-value$. Следовательно верна альтернативная гипотеза: вид спорта влиет на рост спортсмена.